In [2]:
import pandas as pd

In [39]:
raw_df = pd.read_excel('../input/ReportHistory-32012857.xlsx',skiprows=6)
raw_df = raw_df[~raw_df['Commission'].isnull()]
time_config = {
     "format":'%Y.%m.%d %H:%M:%S', 
     "errors":'coerce'
}
# raw_df['Time.1'] = pd.to_datetime(raw_df['Time.1'], format='%Y.%m.%d %H:%M:%S', errors='coerce')
raw_df["Open Time"] = pd.to_datetime(raw_df["Time"],**time_config)
raw_df["Close Time"] = pd.to_datetime(raw_df["Time.1"],**time_config)
raw_df = raw_df.dropna(subset=['Time.1'])
raw_df["Volume"] = pd.to_numeric(raw_df["Volume"], errors="coerce")

# Drop rows where Volume is NaN (invalid)
raw_df = raw_df[raw_df["Volume"].notna()]
raw_df = raw_df.rename(columns={
    "Price": "Open Price",
    "Price.1": "Close Price",
    "Profit": "Profit USD"
})
dtype_mapping = {

    "Profit USD":'int64'
}
raw_df = raw_df.astype(dtype_mapping)
# Drop irrelevant or unnamed columns
df = raw_df[["Open Time", "Close Time", "Symbol", "Type", "Volume", "Open Price", "Close Price", "T / P", "Profit USD"]]


c:\Users\pawin\.pyenv\pyenv-win\versions\3.8.10\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



In [40]:
df

,Open Time,Close Time,Symbol,Type,Volume,Open Price,Close Price,T / P,Profit USD
0,2025-03-24 13:29:47,2025-03-25 08:23:12,XAUUSDc,buy,1.0,3033.09,3019.35,3036.09,-1374
1,2025-03-24 13:31:55,2025-03-25 08:23:12,XAUUSDc,buy,1.0,3030.08,3019.4,3033.08,-1068
2,2025-03-24 14:08:14,2025-03-25 08:23:12,XAUUSDc,buy,1.0,3026.99,3019.37,3029.99,-762
3,2025-03-24 16:27:47,2025-03-25 08:23:11,XAUUSDc,buy,1.0,3023.97,3019.37,3026.97,-460
4,2025-03-24 17:23:20,2025-03-25 08:23:11,XAUUSDc,buy,1.0,3020.9,3019.36,3023.9,-154
...,...,...,...,...,...,...,...,...,...
2606,2025-04-23 05:59:32,2025-04-23 05:59:40,XAUUSDc,buy,2.0,3354.56,3354.63,3357.56,14
2607,2025-04-23 06:12:37,2025-04-23 06:19:28,XAUUSDc,buy,2.0,3336.51,3339.51,3339.51,600
2608,2025-04-23 06:13:00,2025-04-23 06:14:44,XAUUSDc,buy,2.0,3333.36,3336.36,3336.36,600
2609,2025-04-23 06:13:17,2025-04-23 06:13:47,XAUUSDc,buy,2.0,3329.99,3333.29,3332.99,660


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2611 entries, 0 to 2610
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Open Time    2611 non-null   datetime64[ns]
 1   Close Time   2611 non-null   datetime64[ns]
 2   Symbol       2611 non-null   object        
 3   Type         2611 non-null   object        
 4   Volume       2611 non-null   float64       
 5   Open Price   2611 non-null   object        
 6   Close Price  2611 non-null   object        
 7   T / P        2610 non-null   object        
 8   Profit USD   2611 non-null   int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 204.0+ KB


In [42]:
import plotly.express as px


In [27]:
fig2 = px.scatter(
    df,
    x="Open Time",
    y="Open Price",
    size="Volume",
    color="Type",
    title="Trade Entry Prices",
    hover_data=["Close Price", "Profit USD"]
)
fig2.show()


In [54]:
import plotly.express as px
import pandas as pd

# Example DataFrame cleanup if not already done
df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
df = df[df["Volume"].notna()]

# Melt the dataframe to show both Open and Close price as individual points
df_melted = df.melt(
    id_vars=["Open Time", "Close Time", "Type", "Profit USD"],
    value_vars=["Open Price", "Close Price"],
    var_name="Price Type",
    value_name="Price"
)

# Use Open Time for both points to align them visually
df_melted["Time"] = df_melted.apply(
    lambda row: row["Open Time"] if row["Price Type"] == "Open Price" else row["Close Time"],
    axis=1
)

# Color profit: positive = green, negative = red
df_melted["Profit Color"] = df_melted["Profit USD"].apply(
    lambda x: "green" if x >= 0 else "red")
df_melted["Profit Size"] = df_melted["Profit USD"].apply(
    lambda x: abs(x))

fig = px.scatter(
    df_melted,
    x="Time",
    y="Price",
    color="Profit USD",
    color_continuous_scale='YlGn',
    color_continuous_midpoint=0,  # This centers the color scale at 0
    range_color=[-100, 100],           # Set min/max of color range
    size="Profit Size",
    # color_continuous_scale=["red", "orange", "green"],
    symbol="Type",
    facet_col="Type",
    title="Entry and Exit Prices by Trade",
    hover_data=["Profit USD", "Type"]
)

fig.update_layout(height=500, xaxis_title="Time", yaxis_title="Price")
fig.show()

In [35]:
df_melted

,Open Time,Close Time,Type,Profit USD,Price Type,Price,Time,Profit Color
0,2025-03-24 13:29:47,2025-03-25 08:23:12,buy,-1374,Open Price,3033.09,2025-03-24 13:29:47,red
1,2025-03-24 13:31:55,2025-03-25 08:23:12,buy,-1068,Open Price,3030.08,2025-03-24 13:31:55,red
2,2025-03-24 14:08:14,2025-03-25 08:23:12,buy,-762,Open Price,3026.99,2025-03-24 14:08:14,red
3,2025-03-24 16:27:47,2025-03-25 08:23:11,buy,-460,Open Price,3023.97,2025-03-24 16:27:47,red
4,2025-03-24 17:23:20,2025-03-25 08:23:11,buy,-154,Open Price,3020.9,2025-03-24 17:23:20,red
...,...,...,...,...,...,...,...,...
5217,2025-04-23 05:59:32,2025-04-23 05:59:40,buy,14,Close Price,3354.63,2025-04-23 05:59:40,green
5218,2025-04-23 06:12:37,2025-04-23 06:19:28,buy,600,Close Price,3339.51,2025-04-23 06:19:28,green
5219,2025-04-23 06:13:00,2025-04-23 06:14:44,buy,600,Close Price,3336.36,2025-04-23 06:14:44,green
5220,2025-04-23 06:13:17,2025-04-23 06:13:47,buy,660,Close Price,3333.29,2025-04-23 06:13:47,green
